In [6]:
!pip install keras-cv --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 39.8 MB/s eta 0:00:00


In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import keras
import keras_cv
from tensorflow.keras import layers
import tensorflow as tf



In [11]:
img_size = 224
batch = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/KNI Own Materials/Research Materials/Blind People/Vision Research /BD Coin [Classification]/train",
    image_size=(img_size, img_size),
    batch_size=batch,
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/KNI Own Materials/Research Materials/Blind People/Vision Research /BD Coin [Classification]/valid",
    image_size=(img_size, img_size),
    batch_size=batch,
    label_mode='categorical'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/KNI Own Materials/Research Materials/Blind People/Vision Research /BD Coin [Classification]/test",
    image_size=(img_size, img_size),
    batch_size=batch,
    label_mode='categorical'
)

class_names = train_ds.class_names
num_classes = len(class_names)


Found 5843 files belonging to 6 classes.
Found 1708 files belonging to 6 classes.
Found 1718 files belonging to 6 classes.


In [13]:
class PatchExtractor(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID"
        )
        patch_dim = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dim])
        return patches


In [9]:
import keras_cv
from keras_cv.models.backbones import ViTBackbone # Corrected import path

vit = ViTBackbone(
    input_shape=(224, 224, 3),
    variant='vit_base_patch16_224', # A common ViT variant, equivalent to ViTBase16
    include_preprocessing=False # Preprocessing will be handled by the data_augmentation layer or explicitly
)

vit.trainable = False   # Freeze ViT for transfer learning

AttributeError: module 'keras_cv.api.models' has no attribute 'VisionTransformerClassifier'

In [4]:
inputs = keras.Input(shape=(224, 224, 3))

x = data_augmentation(inputs)
# Preprocessing for ViTBackbone is often just normalization, removing incorrect keras.applications.vit.preprocess_input
x = vit(x)

x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)

model = keras.Model(inputs, outputs)

model.summary()

AttributeError: module 'keras.applications' has no attribute 'vit'

In [14]:
def vit_encoder(num_patches, embedding_dim, num_heads, mlp_dim):
    inputs = layers.Input(shape=(num_patches, embedding_dim))

    x = layers.LayerNormalization()(inputs)
    x = layers.MultiHeadAttention(num_heads, key_dim=embedding_dim)(x, x)
    x = layers.Add()([x, inputs])  # residual

    y = layers.LayerNormalization()(x)
    y = layers.Dense(mlp_dim, activation='gelu')(y)
    y = layers.Dense(embedding_dim)(y)

    outputs = layers.Add()([x, y])  # residual
    return keras.Model(inputs, outputs)


In [15]:
patch_size = 16
num_patches = (img_size // patch_size) ** 2
embedding_dim = 64
num_heads = 4
mlp_dim = 128
num_layers = 8   # increase for more parameters

inputs = keras.Input(shape=(img_size, img_size, 3))

patches = PatchExtractor(patch_size)(inputs)
encoded_patches = layers.Dense(embedding_dim)(patches)

positions = tf.range(start=0, limit=num_patches, delta=1)
pos_embed = layers.Embedding(input_dim=num_patches, output_dim=embedding_dim)(positions)
encoded_patches += pos_embed

for _ in range(num_layers):
    encoder = vit_encoder(num_patches, embedding_dim, num_heads, mlp_dim)
    encoded_patches = encoder(encoded_patches)

representation = layers.LayerNormalization()(encoded_patches)
representation = layers.Flatten()(representation)
representation = layers.Dense(256, activation='relu')(representation)
representation = layers.Dropout(0.3)(representation)

outputs = layers.Dense(num_classes, activation='softmax')(representation)

model = keras.Model(inputs, outputs)
model.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ patch_extractor                 │ (None, None, 768)      │             0 │
│ (PatchExtractor)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 64)       │        49,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ add (Add)                       │ (None, 196, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_1 (Functional)       │ (None, 196, 64)        │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_2 (Functional)       │ (None, 196, 64)        │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_3 (Functional)       │ (None, 196, 64)        │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_4 (Functional)       │ (None, 196, 64)        │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_5 (Functional)       │ (None, 196, 64)        │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_6 (Functional)       │ (None, 196, 64)        │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_7 (Functional)       │ (None, 196, 64)        │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_8 (Functional)       │ (None, 196, 64)        │        83,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_16          │ (None, 196, 64)        │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 256)            │     3,211,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,928,006 (14.98 MB)

 Trainable params: 3,928,006 (14.98 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 1107s 6s/step - accuracy: 0.2797 - loss: 2.0051 - val_accuracy: 0.3560 - val_loss: 1.5912
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 579s 3s/step - accuracy: 0.4160 - loss: 1.5248 - val_accuracy: 0.5217 - val_loss: 1.2034
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 575s 3s/step - accuracy: 0.5678 - loss: 1.1359 - val_accuracy: 0.5878 - val_loss: 1.1156
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 643s 3s/step - accuracy: 0.6762 - loss: 0.8804 - val_accuracy: 0.6674 - val_loss: 0.9414
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 605s 3s/step - accuracy: 0.7760 - loss: 0.6458 - val_accuracy: 0.7652 - val_loss: 0.6570
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 602s 3s/step - accuracy: 0.8371 - loss: 0.4629 - val_accuracy: 0.7617 - val_loss: 0.6670
Epoch 7/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 619s 3s/step - accuracy: 0.8764 - loss: 0.3616 - val_accuracy: 0.7652 - val_loss: 0.8664
Epoch 8/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 585s 3s/step - accuracy: 0.9221 - loss: 0.2441 - val_acc

In [17]:
!pip install scikit-learn


In [18]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_true = []
y_pred = []

# Iterate over the test dataset
for images, labels in test_ds:
    preds = model.predict(images)
    preds = np.argmax(preds, axis=1)
    true = np.argmax(labels.numpy(), axis=1)

    y_pred.extend(preds)
    y_true.extend(true)

y_true = np.array(y_true)
y_pred = np.array(y_pred)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 954ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 942ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 995ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 970ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 964ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 955ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 941ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 932ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 964ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 990ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 948ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 944ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 984ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 955ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 993ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 944ms/step
1/1 ━━━━━━━━━━━━

In [19]:
print(classification_report(y_true, y_pred, target_names=class_names))


              precision    recall  f1-score   support

          1b       0.65      0.93      0.77       100
          1f       0.63      0.96      0.76       130
          2b       0.57      0.92      0.70       100
          2f       0.85      0.97      0.91       265
          5b       0.82      0.96      0.88       100
          5f       1.00      0.78      0.87      1023

    accuracy                           0.85      1718
   macro avg       0.75      0.92      0.82      1718
weighted avg       0.89      0.85      0.85      1718

